In [ ]:
archive_name  = ""
raw_bucket    = ""
output_bucket = ""

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, isnan, sum, when, isnull, regexp_replace
# Importa os tipos de dados necessários para o cast correto
from pyspark.sql.types import LongType, IntegerType, DoubleType

# Configurações do Spark (MANTIDAS)
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("UnificarUPA").getOrCreate()

# ======================
# ARQUIVOS E CAMINHOS
# ======================
# s3_prefix = 's3a://bucket-raw-upa-connect-sofh/arquivos/'

# arquivos_sensor = [
#     '2025_10_13.csv',
#     '2025_10_12.csv',
#     '2025_10_11.csv',
#     '2025_10_10.csv',
#     '2025_10_09.csv'
# ]


# ----------------------------------------------------
# ALTERAÇÃO: 'fk_paciente' REMOVIDA da lista de colunas finais
# ----------------------------------------------------
colunas_finais = ['data_hora', 'valor', 'fk_sensor', 'fk_unid_medida', 'fk_upa']  


# ======================
# LEITURA DOS ARQUIVOS
# ======================

# sensor_paths = [s3_prefix + f for f in arquivos_sensor]
sensor_paths = f's3a://{raw_bucket}/{archive_name}'


sensor_df = spark.read.option('header', 'true').option('delimiter', ',').csv(sensor_paths)
# Leitura do atendimento_df omitida ou ajustada
atendimento_df = spark.createDataFrame([], schema=sensor_df.schema)


# Adicionar colunas que não existam em algum dos DataFrames
for col_name in colunas_finais:
    if col_name not in sensor_df.columns:
        sensor_df = sensor_df.withColumn(col_name, lit(None))
    if col_name not in atendimento_df.columns:
        atendimento_df = atendimento_df.withColumn(col_name, lit(None))

# Selecionar e ordenar as colunas
sensor_df = sensor_df.select(colunas_finais)


# ==========================================
# TRATAMENTO DE OUTLIERS E NULOS (REVISADA)
# ==========================================

print(f"Total de linhas antes do tratamento: {sensor_df.count()}")

# 1. Criar uma coluna temporária 'valor_limpo'
#    - Primeiro, substitui vírgula (,) por ponto (.) para padronização
#    - Segundo, remove todos os caracteres que NÃO sejam dígitos (0-9) ou o ponto (.)
sensor_df = sensor_df.withColumn(
    "valor_limpo",
    regexp_replace(
        regexp_replace(col("valor"), ",", "."), # 1. Substitui vírgula por ponto
        "[^0-9\\.]", ""                         # 2. Remove lixo restante (seu regex)
    )
)

# 1.1. Fazer o cast condicional, substituindo a coluna 'valor' original
#    - se fk_sensor == 1, converte para Long (inteiro, sem decimais)
#    - se fk_sensor == 2 (ou qualquer outro valor), converte para Double (mantém decimais)
sensor_df = sensor_df.withColumn(
    "valor", # Substitui a coluna 'valor' original
    when(col("fk_sensor") == 1,
         col("valor_limpo").cast(LongType()))
    .otherwise(
         col("valor_limpo").cast(DoubleType()))
).drop("valor_limpo") # Remove a coluna temporária que não é mais necessária


# 1.2 Cast das outras colunas
sensor_df = sensor_df.withColumn("fk_sensor", col("fk_sensor").cast(IntegerType()))
sensor_df = sensor_df.withColumn("fk_unid_medida", col("fk_unid_medida").cast(IntegerType()))

# --- Condições de EXCLUSÃO ---
# NOTA: O cast para LongType (e não double) garante que números grandes inteiros sejam comparados corretamente.

# 1. fk_sensor = 1 (Geral): apagar linhas em que valor é MAIOR que 300, e MENOR que 0
# CÂMERA VISÃO COMPUTACIONAL
condicao_exclusao_1 = (
    (col("fk_sensor") == 1) &
    (
        (col("valor") > 300) |               # Maior que 300
        (col("valor") < 0)                   # Menor que 0
    )
)

# 2. fk_sensor = 2 e fk_unid_medida = 1: apagar linhas em que valor é MAIOR que 37 e MENOR que 12
# TEMPERATURA DO AMBIENTE
condicao_exclusao_2 = (
    (col("fk_sensor") == 2) &
    (col("fk_unid_medida") == 1) &
    (
        (col("valor") > 37) |              
        (col("valor") < 12)
    )
)

# 3. fk_sensor = 2 e fk_unid_medida = 2: apagar linhas em que valor é MAIOR que 95 e MENOR que 10
# UMIDADE DO AMBIENTE
condicao_exclusao_3 = (
    (col("fk_sensor") == 2) &
    (col("fk_unid_medida") == 2) &
    (
        (col("valor") > 95) |              
        (col("valor") < 10)
    )
)

# Combinação das condições: Uma linha deve ser APAGADA se atender a QUALQUER uma das condições de OUTLIER
condicao_outlier_total = condicao_exclusao_1 | condicao_exclusao_2 | condicao_exclusao_3

# Condição final de exclusão (Outlier OU valor que se tornou NULL/NaN)
# Se você quer APAGAR os outliers E os valores que falharam no cast (NULL/NaN), use a lógica abaixo:
condicao_exclusao_total = (
    condicao_outlier_total         # Exclui outliers numéricos (agora precisos)
    | col("valor").isNull()        # Exclui valores estritamente NULL (inclui falha de cast)
    | isnan(col("valor"))          # Exclui NaN (menos comum com LongType, mas boa prática)
)

# Aplicar o filtro: Manter apenas as linhas onde a condição de exclusão (outlier ou nulo) NÃO é verdadeira.
sensor_df = sensor_df.filter(~condicao_exclusao_total)

# ======================
# RESULTADOS
# ======================
sensor_df.show(10)
print(f"Total de linhas após o tratamento: {sensor_df.count()}")

# Opcionalmente, pode descomentar a seção abaixo se quiser o cálculo de nulos, mas
# a contagem de nulos na coluna 'valor' agora será zero, pois eles foram excluídos.

# # ==========================================
# # CONTAGEM ESPECÍFICA PARA NULL NA COLUNA 'VALOR'
# # ==========================================

# # 1. Cria a expressão de agregação: soma 1 se o valor for estritamente NULL.
# total_nulos_valor = sum(
#      when(col("valor").isNull(), 1).otherwise(0)
# ).alias("total_nulos_estritos_valor")

# # 2. Aplica a agregação e coleta o resultado
# resultado = sensor_df.agg(total_nulos_valor).collect()

# # 3. Exibe o resultado de forma limpa
# contagem = resultado[0]["total_nulos_estritos_valor"]

# print(f"\n=========================================================")
# print(f"Contagem de registros com valor NULL na coluna 'valor': {contagem}")
# print(f"=========================================================")

# # Você também pode simplesmente mostrar o DataFrame resultante da agregação:
# sensor_df.agg(total_nulos_valor).show()

In [ ]:
%pip install boto3

import boto3
from io import StringIO, BytesIO
from botocore.exceptions import ClientError
import pandas as pd

FINAL_OUTPUT_DIR = f"s3a://{output_bucket}/"
FINAL_FILENAME = "tabela_sensores_tratada.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"
FINAL_FILE_PATH = f"{FINAL_OUTPUT_DIR}{FINAL_FILENAME}"

s3 = boto3.client("s3")

try:
    # 🧾 Verifica se o arquivo existe
    s3.head_object(Bucket=output_bucket, Key=FINAL_FILENAME)

    print("📥 Arquivo existente encontrado. Lendo do S3...")

    # ⚙️ Lê o CSV existente direto do S3 (Spark)
    df_existente = (
        spark.read
        .option('header', 'true')
        .option('delimiter', ',')
        .csv(f"s3a://{output_bucket}/{FINAL_FILENAME}")
    )

    # 🔗 Concatena (Spark -> Pandas)
    df_existente_pd = df_existente.toPandas()
    sensor_pd = sensor_df.toPandas()  # sensor_df também é Spark, então converte
    df_final = pd.concat([df_existente_pd, sensor_pd], ignore_index=True)

    print("✅ Arquivo existente atualizado com novos dados.")

except ClientError as e:
    if e.response["Error"]["Code"] == "404":
        print("🚫 Arquivo não encontrado. Criando novo arquivo no bucket.")
        df_final = sensor_df.toPandas()
    else:
        raise

# 📤 Salva no S3 via boto3
csv_buffer = StringIO()
df_final.to_csv(csv_buffer, index=False)

s3.put_object(
    Bucket=output_bucket,
    Key=FINAL_FILENAME,
    Body=csv_buffer.getvalue()
)

print(f"✅ Arquivo '{FINAL_FILENAME}' salvo com sucesso no bucket '{output_bucket}'.")
spark.stop()